<a href="https://colab.research.google.com/github/aravind-chilakamarri/CS5001-Fundamentals-of-Data-Engineering/blob/main/Project_6_DB_SQL_part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MMA Bootcamp - Intro to SQL

## 1. Preparation

We start by setting up a connection to our sample database, Northwind DB.

In [ ]:
# download Northwind SQLite DB
!wget https://tdmdal.github.io/mma-sql/data/northwind.sqlite3

--2023-10-11 00:18:46--  https://tdmdal.github.io/mma-sql/data/northwind.sqlite3
Resolving tdmdal.github.io (tdmdal.github.io)... 185.199.111.153, 185.199.110.153, 185.199.108.153, ...
Connecting to tdmdal.github.io (tdmdal.github.io)|185.199.111.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 804864 (786K) [application/octet-stream]
Saving to: ‘northwind.sqlite3’

northwind.sqlite3   100%[===================>] 786.00K  --.-KB/s    in 0.03s   

2023-10-11 00:18:46 (22.2 MB/s) - ‘northwind.sqlite3’ saved [804864/804864]



In [ ]:
# load the SQL magic extension
# https://github.com/catherinedevlin/ipython-sql
# this extension allows us to connect to DBs and issue SQL command
%load_ext sql

# now we can use the magic extension to connect to our SQLite DB
# use %sql to write an inline SQL command
# use %%sql to write SQL commands in a cell
%sql sqlite:///northwind.sqlite3

After connecting to the Northwind DB, let's first check what database engine we are using.

In [ ]:
%%sql
SELECT sqlite_version();

 * sqlite:///northwind.sqlite3
Done.


sqlite_version()
3.37.2


We are using [SQLite 3](https://www.sqlite.org/index.html). It's a small and fast Relational Database engine. SQLite DB is self-contained: one database is just a file. Believe it or not, SQLite is the most used DB engine in the world. It's built into millions of mobile phones (e.g. Android or iOS phones) and it's used by many popular apps to store data.  

## 2. A quick look at Northwind DB

### How many tables does this DB have?

A database may contain many tables. Let's see how many tables we have in the Northwind DB.

Every SQLite database has a special table called `sqlite_master`. It contains a master listing of all database objects (tables, indexes, etc.) in the database and the SQL used to create each object. We can query this table to find out how many tables (excluding `sqlite_master`) we have in our Northwind DB.

We will use SQL keywords `SELECT...FROM...WHERE`. Don't worry about them for now. We will soon learn those keywords.

In [ ]:
%%sql
SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%';

 * sqlite:///northwind.sqlite3
Done.


name
Suppliers
Shippers
Employees
Customers
Orders
Categories
Products
OrderDetails


Note that special tables in SQLite start with `sqlite_`. Those are reserved tables for SQLite engine system use. Above we filtered out those tables. In most cases, we shouldn't touch those special tables.

### How does the Categories table look like?

A table is made up of one or more columns/fields. Let's take a look a the Categories table (column names, types, etc.).

In [ ]:
%sql PRAGMA table_info([Categories]);

 * sqlite:///northwind.sqlite3
Done.


cid,name,type,notnull,dflt_value,pk
0,CategoryID,INTEGER,1,None,1
1,CategoryName,nvarchar(15),1,None,0
2,Description,ntext,0,None,0
3,Picture,image,0,None,0


### How about the relationships between the tables?

We present the relationships between the tables using an Entity Relationship (ER) diagram.

<img src="https://tdmdal.github.io/mma-sql/images/NW_ER.png" alt="ER Diagram" width="836" height="605" />

In the above ER diagram, the tiny vertical key icon indicates a column is a primary key. A primary key is a column (or set of columns) whose values uniquely identify every row in a table. For example, `OrderID` is the primary key in the `Orders` table, and `OrderID` and `ProductID` (combined) is the primary key in the `OrderDetails` table.

The relationship icon (a line with a horizontal key at one end and an infinite symbol at the other end) indicates a foreign key constraint and a one-to-many relationship. A foreign key is a column (or set of columns) in one table whose values uniquely identify a row of another table or the same table. A foreign key mostly refers to a primary key in another table. A foreign key constraint requires that the constrained column contain only values from the primary key column of the other table. For example `CustomerID` in the `Orders` table is a foreign key that refers to the `CustomerID` primary key in the `Customers` table, and it can only contain values that exist in the `CustomerID` column of the `Customers` table.

In addition, it happens that every foreign key constraint in the Northwind DB establishes a one-to-many relationship, i.e. a row from one table can have multiple matching rows in another table. For example, one row from the `Customers` table can match multiple rows in the `Orders` table (via `CustomerID`). This makes sense as one customer can place more than one orders. (Another common relationship a foreign key constraint can establish is the one-to-one relationship.)

|logo|meaning|
|:------:|:------:|
|![key logo](https://tdmdal.github.io/mma-sql/images/key_vertical.png "key logo")|primary key|
|![foreign key constraint](https://tdmdal.github.io/mma-sql/images/relationship.png "foreign key constraint")|one-to-many foreign key contraint|

**Question**: Why do we need foreign key constraints? (Discussion)

You can find out relationships between tables by querying the `sqlite_master` table. See the code below.

Of course, querying the `sqlite_master` table to find out relationships between tables is specific to SQLite. We would do it differently if we use other database management systems. For example, if we use MySQL system, we would query the `key_column_usage` table in the `information_schema` DB. See this stackoverflow [question](https://stackoverflow.com/questions/20855065/how-to-find-all-the-relations-between-all-mysql-tables).

Alternatively, some SQL client tools can generate ER diagram for you. The above diagram is generated by SQL Server Management Studio (SSMS). The Northwind DB is a sample DB originally shipped with Microsoft SQL Server.

In [ ]:
%%sql
-- find out relationships between tables using SQL command
SELECT sql
FROM sqlite_master
WHERE name = "Orders"

 * sqlite:///northwind.sqlite3
Done.


sql
"CREATE TABLE [Orders] ( [OrderID] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, [CustomerID] nchar(5) NOT NULL COLLATE NOCASE, [EmployeeID] int NULL, [OrderDate] datetime NULL, [RequiredDate] datetime NULL, [ShippedDate] datetime NULL, [ShipVia] int NULL, [Freight] money DEFAULT 0 NULL, [ShipName] nvarchar(40) NULL COLLATE NOCASE, [ShipAddress] nvarchar(60) NULL COLLATE NOCASE, [ShipCity] nvarchar(15) NULL COLLATE NOCASE, [ShipRegion] nvarchar(15) NULL COLLATE NOCASE, [ShipPostalCode] nvarchar(10) NULL COLLATE NOCASE, [ShipCountry] nvarchar(15) NULL COLLATE NOCASE, CONSTRAINT [FK_Orders_Customers] FOREIGN KEY ([CustomerID]) REFERENCES [Customers] ([CustomerID]) ON DELETE NO ACTION ON UPDATE NO ACTION, CONSTRAINT [FK_Orders_Employees] FOREIGN KEY ([EmployeeID]) REFERENCES [Employees] ([EmployeeID]) ON DELETE NO ACTION ON UPDATE NO ACTION, CONSTRAINT [FK_Orders_Shippers] FOREIGN KEY ([ShipVia]) REFERENCES [Shippers] ([ShipperID]) ON DELETE NO ACTION ON UPDATE NO ACTION)"


## Part 3. Join Tables

1. Inner join: `SELECT...FROM...INNER JOIN...ON...`
2. Left join:  `SELECT...FROM...LEFT JOIN...ON...`
3. Other joins.

### Ex.16* Show products and their associated suppliers

Display the `ProductID`, `ProductName`, and the `CompanyName` of the Supplier. Sort by `ProductID`.

We can use the `WHERE` keyword.

In [ ]:
%%sql
-- start your code here
SELECT p.ProductID, p.ProductName, s.CompanyName
FROM Products p, Suppliers s
WHERE p.SupplierID = s.SupplierID
ORDER BY p.ProductID;

 * sqlite:///northwind.sqlite3
Done.


ProductID,ProductName,CompanyName
1,Chai,Exotic Liquids
2,Chang,Exotic Liquids
3,Aniseed Syrup,Exotic Liquids
4,Chef Anton's Cajun Seasoning,New Orleans Cajun Delights
5,Chef Anton's Gumbo Mix,New Orleans Cajun Delights
6,Grandma's Boysenberry Spread,Grandma Kelly's Homestead
7,Uncle Bob's Organic Dried Pears,Grandma Kelly's Homestead
8,Northwoods Cranberry Sauce,Grandma Kelly's Homestead
9,Mishi Kobe Niku,Tokyo Traders
10,Ikura,Tokyo Traders


We can also use the `(INNER) JOIN` keyword.

In [ ]:
%%sql
-- start your code here
SELECT p.ProductID, p.ProductName, s.CompanyName
FROM Products p
INNER JOIN Suppliers s ON p.SupplierID = s.SupplierID
ORDER BY p.ProductID;

 * sqlite:///northwind.sqlite3
Done.


ProductID,ProductName,CompanyName
1,Chai,Exotic Liquids
2,Chang,Exotic Liquids
3,Aniseed Syrup,Exotic Liquids
4,Chef Anton's Cajun Seasoning,New Orleans Cajun Delights
5,Chef Anton's Gumbo Mix,New Orleans Cajun Delights
6,Grandma's Boysenberry Spread,Grandma Kelly's Homestead
7,Uncle Bob's Organic Dried Pears,Grandma Kelly's Homestead
8,Northwoods Cranberry Sauce,Grandma Kelly's Homestead
9,Mishi Kobe Niku,Tokyo Traders
10,Ikura,Tokyo Traders


### Ex.17 Show all orders and shippers with OrderID less than 10255.

Display `OrderID`, `OrderDate` (date only), and `CompanyName` of the Shipper, and sort by `OrderID`. In addition, only display rows with `OrderID < 10255`.

In [ ]:
%%sql
SELECT o.OrderID, DATE(o.OrderDate) AS OrderDate, s.CompanyName
FROM Orders o
INNER JOIN Shippers s ON o.ShipVia = s.ShipperID
WHERE o.OrderID < 10255
ORDER BY o.OrderID;

 * sqlite:///northwind.sqlite3
Done.


OrderID,OrderDate,CompanyName
10248,2014-07-04,Federal Shipping
10249,2014-07-05,Speedy Express
10250,2014-07-08,United Package
10251,2014-07-08,Speedy Express
10252,2014-07-09,United Package
10253,2014-07-10,United Package
10254,2014-07-11,United Package


### Ex.18* Find total number of products in each category

In your result, display `CategoryName` and total number of product.

In [ ]:
%%sql
-- start your code here
SELECT c.CategoryName, COUNT(p.ProductID) AS total_number_of_products
FROM Categories c
JOIN Products p ON c.CategoryID = p.CategoryID
GROUP BY c.CategoryName;


 * sqlite:///northwind.sqlite3
Done.


CategoryName,total_number_of_products
Beverages,12
Condiments,12
Confections,13
Dairy Products,10
Grains/Cereals,7
Meat/Poultry,6
Produce,5
Seafood,12


In [ ]:
%%sql

SELECT * FROM Orders LIMIT 5

 * sqlite:///northwind.sqlite3
Done.


OrderID,CustomerID,EmployeeID,OrderDate,RequiredDate,ShippedDate,ShipVia,Freight,ShipName,ShipAddress,ShipCity,ShipRegion,ShipPostalCode,ShipCountry
10248,VINET,5,2014-07-04 08:00:00.000,2014-08-01 00:00:00.000,2014-07-16 00:00:00.000,3,32.38,Vins et alcools Chevalier,59 rue de l'Abbaye,Reims,None,51100,France
10249,TOMSP,6,2014-07-05 04:00:00.000,2014-08-16 00:00:00.000,2014-07-10 00:00:00.000,1,11.61,Toms Spezialitäten,Luisenstr. 48,Münster,None,44087,Germany
10250,HANAR,4,2014-07-08 15:00:00.000,2014-08-05 00:00:00.000,2014-07-12 00:00:00.000,2,65.83,Hanari Carnes,"Rua do Paço, 67",Rio de Janeiro,RJ,05454-876,Brazil
10251,VICTE,3,2014-07-08 14:00:00.000,2014-08-05 00:00:00.000,2014-07-15 00:00:00.000,1,41.34,Victuailles en stock,"2, rue du Commerce",Lyon,None,69004,France
10252,SUPRD,4,2014-07-09 01:00:00.000,2014-08-06 00:00:00.000,2014-07-11 00:00:00.000,2,51.3,Suprêmes délices,"Boulevard Tirou, 255",Charleroi,None,B-6000,Belgium


In [ ]:
%%sql

SELECT * FROM OrderDetails LIMIT 5

 * sqlite:///northwind.sqlite3
Done.


OrderID,ProductID,UnitPrice,Quantity,Discount
10248,11,14,12,0.0
10248,42,9.8,10,0.0
10248,72,34.8,5,0.0
10249,14,18.6,9,0.0
10249,51,42.4,40,0.0


### Ex.19 Show all orders with values greater than $12,000 and are placed in 2016?

In your result, display `OrderID` and total value of the order (ignore `Discount`).

In [16]:
%%sql
-- start your code SELECT OrderID, (UnitPrice * Quantity) AS TotalValue
SELECT od.OrderID, SUM(od.UnitPrice * od.Quantity) AS TotalValue
FROM OrderDetails od
JOIN Orders o ON od.OrderID = o.OrderID
WHERE strftime('%Y', o.OrderDate) = '2016'
GROUP BY od.OrderID
HAVING TotalValue > 12000;

 * sqlite:///northwind.sqlite3
Done.


OrderID,TotalValue
10865,17250.0
10981,15810.0
11030,16321.900000000001


### Ex.20* What products has Michael Suyama ever sold?

In your result, display `EmployeeID`, `FirstName`, `LastName`, `ProductID` and `ProductName`.

In [21]:
%%sql

SELECT * FROM Employees LIMIT 1

 * sqlite:///northwind.sqlite3
Done.


[(1, 'Davolio', 'Nancy', 'Sales Representative', 'Ms.', '1966-12-08 00:00:00.000', '2010-05-01 00:00:00.000', '507 - 20th Ave. E.\nApt. 2A', 'Seattle', 'WA', '98122', 'USA', '(206) 555-9857', '5467', b'\x15\x1c/\x00\x02\x00\x00\x00\r\x00\x0e\x00\x14\x00!\x00\xff\xff\xff\xffBitmap Image\x00Paint.Picture\x00\x01\x05\x00\x00\x02\x00\x00\x00\x07\x00\x0 ... (80909 characters truncated) ... 0\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x05\x00\x00\x00\x00\x00\x00\xb4\xad\x05\xfe', 'Education includes a BA in psychology from Colorado State University in 1970.  She also completed "The Art of the Cold Call."  Nancy is a member of Toastmasters International.', 2, 'http://accweb/emmployees/davolio.bmp')]

In [22]:
%%sql

SELECT * FROM Orders LIMIT 1

 * sqlite:///northwind.sqlite3
Done.


OrderID,CustomerID,EmployeeID,OrderDate,RequiredDate,ShippedDate,ShipVia,Freight,ShipName,ShipAddress,ShipCity,ShipRegion,ShipPostalCode,ShipCountry
10248,VINET,5,2014-07-04 08:00:00.000,2014-08-01 00:00:00.000,2014-07-16 00:00:00.000,3,32.38,Vins et alcools Chevalier,59 rue de l'Abbaye,Reims,None,51100,France


In [23]:
%%sql

SELECT * FROM OrderDetails LIMIT 1

 * sqlite:///northwind.sqlite3
Done.


OrderID,ProductID,UnitPrice,Quantity,Discount
10248,11,14,12,0.0


In [24]:
%%sql

SELECT * FROM Products LIMIT 1

 * sqlite:///northwind.sqlite3
Done.


ProductID,ProductName,SupplierID,CategoryID,QuantityPerUnit,UnitPrice,UnitsInStock,UnitsOnOrder,ReorderLevel,Discontinued
1,Chai,1,1,10 boxes x 20 bags,18,39,0,10,0


In [18]:
%%sql
-- start your code here
SELECT e.EmployeeID, e.FirstName, e.LastName, od.ProductID, p.ProductName
FROM Employees e
JOIN Orders o ON e.EmployeeID = o.EmployeeID
JOIN OrderDetails od ON o.OrderID = od.OrderID
JOIN Products p ON od.ProductID = p.ProductID
WHERE e.FirstName = 'Michael' AND e.LastName = 'Suyama'
GROUP BY p.ProductID

 * sqlite:///northwind.sqlite3
Done.


EmployeeID,FirstName,LastName,ProductID,ProductName
6,Michael,Suyama,1,Chai
6,Michael,Suyama,2,Chang
6,Michael,Suyama,3,Aniseed Syrup
6,Michael,Suyama,4,Chef Anton's Cajun Seasoning
6,Michael,Suyama,5,Chef Anton's Gumbo Mix
6,Michael,Suyama,7,Uncle Bob's Organic Dried Pears
6,Michael,Suyama,10,Ikura
6,Michael,Suyama,11,Queso Cabrales
6,Michael,Suyama,12,Queso Manchego La Pastora
6,Michael,Suyama,13,Konbu


### Ex.21* Find customers that never placed an order

In [28]:
%%sql
SELECT * FROM customers LIMIT 10

 * sqlite:///northwind.sqlite3
Done.


CustomerID,CompanyName,ContactName,ContactTitle,Address,City,Region,PostalCode,Country,Phone,Fax
ALFKI,Alfreds Futterkiste,Maria Anders,Sales Representative,Obere Str. 57,Berlin,None,12209,Germany,030-0074321,030-0076545
ANATR,Ana Trujillo Emparedados y helados,Ana Trujillo,Owner,Avda. de la Constitución 2222,México D.F.,None,05021,Mexico,(5) 555-4729,(5) 555-3745
ANTON,Antonio Moreno Taquería,Antonio Moreno,Owner,Mataderos 2312,México D.F.,None,05023,Mexico,(5) 555-3932,None
AROUT,Around the Horn,Thomas Hardy,Sales Representative,120 Hanover Sq.,London,None,WA1 1DP,UK,(171) 555-7788,(171) 555-6750
BERGS,Berglunds snabbköp,Christina Berglund,Order Administrator,Berguvsvägen 8,Luleå,None,S-958 22,Sweden,0921-12 34 65,0921-12 34 67
BLAUS,Blauer See Delikatessen,Hanna Moos,Sales Representative,Forsterstr. 57,Mannheim,None,68306,Germany,0621-08460,0621-08924
BLONP,Blondesddsl père et fils,Frédérique Citeaux,Marketing Manager,"24, place Kléber",Strasbourg,None,67000,France,88.60.15.31,88.60.15.32
BOLID,Bólido Comidas preparadas,Martín Sommer,Owner,"C/ Araquil, 67",Madrid,None,28023,Spain,(91) 555 22 82,(91) 555 91 99
BONAP,Bon app',Laurence Lebihan,Owner,"12, rue des Bouchers",Marseille,None,13008,France,91.24.45.40,91.24.45.41
BOTTM,Bottom-Dollar Markets,Elizabeth Lincoln,Accounting Manager,23 Tsawassen Blvd.,Tsawassen,BC,T2F 8M4,Canada,(604) 555-4729,(604) 555-3745


In [42]:
%%sql
SELECT * FROM Orders LIMIT 5

 * sqlite:///northwind.sqlite3
Done.


OrderID,CustomerID,EmployeeID,OrderDate,RequiredDate,ShippedDate,ShipVia,Freight,ShipName,ShipAddress,ShipCity,ShipRegion,ShipPostalCode,ShipCountry
10248,VINET,5,2014-07-04 08:00:00.000,2014-08-01 00:00:00.000,2014-07-16 00:00:00.000,3,32.38,Vins et alcools Chevalier,59 rue de l'Abbaye,Reims,None,51100,France
10249,TOMSP,6,2014-07-05 04:00:00.000,2014-08-16 00:00:00.000,2014-07-10 00:00:00.000,1,11.61,Toms Spezialitäten,Luisenstr. 48,Münster,None,44087,Germany
10250,HANAR,4,2014-07-08 15:00:00.000,2014-08-05 00:00:00.000,2014-07-12 00:00:00.000,2,65.83,Hanari Carnes,"Rua do Paço, 67",Rio de Janeiro,RJ,05454-876,Brazil
10251,VICTE,3,2014-07-08 14:00:00.000,2014-08-05 00:00:00.000,2014-07-15 00:00:00.000,1,41.34,Victuailles en stock,"2, rue du Commerce",Lyon,None,69004,France
10252,SUPRD,4,2014-07-09 01:00:00.000,2014-08-06 00:00:00.000,2014-07-11 00:00:00.000,2,51.3,Suprêmes délices,"Boulevard Tirou, 255",Charleroi,None,B-6000,Belgium


In [30]:
%%sql
-- start your code here
SELECT c.CustomerID, c.ContactName, c.CompanyName
FROM Customers c
LEFT JOIN Orders o ON c.CustomerID = o.CustomerID
WHERE o.CustomerID IS NULL;


 * sqlite:///northwind.sqlite3
Done.


CustomerID,ContactName,CompanyName
FISSA,Diego Roel,FISSA Fabrica Inter. Salchichas S.A.
PARIS,Marie Bertrand,Paris spécialités


Note that the Jupyter Notebook displays Null as None.



### Ex.22* Find customers who never placed an order from Margaret Peacock (EmployeeID 4)

In [39]:
%%sql
-- start your code here
SELECT c.CustomerID, c.ContactName
FROM Customers c
LEFT JOIN Orders o ON c.CustomerID = o.CustomerID
LEFT JOIN Employees e ON o.EmployeeID = e.EmployeeID
WHERE e.EmployeeID IS NULL OR e.EmployeeID <> 4
GROUP BY c.CustomerID, c.ContactName;


 * sqlite:///northwind.sqlite3
Done.


CustomerID,ContactName
ALFKI,Maria Anders
ANATR,Ana Trujillo
ANTON,Antonio Moreno
AROUT,Thomas Hardy
BERGS,Christina Berglund
BLAUS,Hanna Moos
BLONP,Frédérique Citeaux
BOLID,Martín Sommer
BONAP,Laurence Lebihan
BOTTM,Elizabeth Lincoln


### Ex.23* Find high-value customers - 1

We define high-value customers as those who have made at least 1 order with a total value >= $10,000 (ignore discount). We only consider orders placed in year 2016.

In [41]:
%%sql
-- start your code here
SELECT c.CustomerID, c.ContactName, SUM(od.UnitPrice * od.Quantity) AS TotalSpent
FROM Customers c
JOIN Orders o ON c.CustomerID = o.CustomerID
JOIN OrderDetails od ON o.OrderID = od.OrderID
WHERE strftime('%Y', O.OrderDate) = '2016'
GROUP BY c.CustomerID, c.ContactName
HAVING SUM(od.UnitPrice * od.Quantity) >= 10000;

 * sqlite:///northwind.sqlite3
Done.


CustomerID,ContactName,TotalSpent
BOTTM,Elizabeth Lincoln,12227.4
ERNSH,Roland Mendel,42598.9
FOLKO,Maria Larsson,15973.85
GREAL,Howard Snyder,10562.58
HANAR,Mario Pontes,24238.05
HUNGO,Patricia McKenna,22796.340000000004
KOENE,Philip Cramer,20204.95
LINOD,Felipe Izquierdo,10085.6
QUICK,Horst Kloss,40526.99
RATTC,Paula Wilson,21725.600000000002


### Ex.24 Find high-value customers - 2

This time, we define high-value customers as those who have made total order >= $15,000 (ignore discount). We only consider orders placed in year 2016.

In [44]:
%%sql
SELECT c.CustomerID, c.ContactName, SUM(od.UnitPrice * od.Quantity) AS TotalSpent
FROM Customers c
JOIN Orders o ON c.CustomerID = o.CustomerID
JOIN OrderDetails od ON o.OrderID = od.OrderID
WHERE strftime('%Y', O.OrderDate) = '2016'
GROUP BY c.CustomerID, c.ContactName
HAVING SUM(od.UnitPrice * od.Quantity) >= 15000;


 * sqlite:///northwind.sqlite3
Done.


CustomerID,ContactName,TotalSpent
ERNSH,Roland Mendel,42598.9
FOLKO,Maria Larsson,15973.85
HANAR,Mario Pontes,24238.05
HUNGO,Patricia McKenna,22796.340000000004
KOENE,Philip Cramer,20204.95
QUICK,Horst Kloss,40526.99
RATTC,Paula Wilson,21725.600000000002
SAVEA,Jose Pavarotti,42806.25
WHITC,Karl Jablonski,15278.9
